### Exploration with Kaggle data

Aim is to validate model choice, and that entire pipeline from data ingestion to prediction works.

Data from: https://www.kaggle.com/datasets/harrisonlou/imu-glove/data <br>
Note that data files are not standardised, so I had to do windowing.

#### Results

| Model | Accuracy / % |
|---|---|
| RNN + LSTM (with mean, std dev) | 100 |
| RNN + LSTM | 100 |
| 1D CNN | 99.2 |
| MLP| 99.0 |

In [1]:
import os
import torch
import torch.optim as optim
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from model_definition import LSTMClassifier, CNN1DClassifier, MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [2]:
class WindowedCSVDataset(Dataset):
    def __init__(self, csv_dir, labels_dict, statistical_processing=False, window_size=25, stride=5):
        self.samples = []
      
        for file_name, label in labels_dict.items():
            file_path = os.path.join(csv_dir, file_name)
            if not os.path.exists(file_path):
                continue

            df = pd.read_csv(file_path)
            x = torch.tensor(df.iloc[:, 1:].values, dtype=torch.float32)  # [T, F]

            T, F = x.shape

            # handle short CSVs
            if T < window_size:
                pad = torch.zeros(window_size - T, F)
                window = torch.cat([x, pad], dim=0)  # [window_size, F]
                self.samples.append((window, torch.tensor(label, dtype=torch.long)))

            for start in range(0, T - window_size + 1, stride):
                window = x[start:start + window_size] # [25, 44]
                
                if statistical_processing:
                    # Calculate stats for this specific window
                    w_mean = window.mean(dim=0) # [44]
                    w_std = window.std(dim=0)   # [44]
                    
                    # Expand stats to match window length [25, 44]
                    mean_feat = w_mean.unsqueeze(0).expand(window_size, -1)
                    std_feat = w_std.unsqueeze(0).expand(window_size, -1)
                    
                    # Concat: [25, 44 + 44 + 44] -> [25, 132]
                    window = torch.cat([window, mean_feat, std_feat], dim=-1)
                
                self.samples.append((window, torch.tensor(label, dtype=torch.long)))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]
    
def create_dataloader(dataset, csv_dir, labels_dict, statistical_processing=False, batch_size=4):
    data = dataset(csv_dir, labels_dict, statistical_processing)
    return DataLoader(data, batch_size=batch_size, shuffle=True)

In [3]:
imu_data_path = "data/kaggle_imu/data"
imu_label_path = "data/kaggle_imu/label"        # labels are [0, 1, 2, 3, 4, 5, 6, 10->7]

def get_labels(label_folder_path):
    labels = dict()

    for file_name in os.listdir(label_folder_path):
        file_path = os.path.join(label_folder_path, file_name)
        data = pd.read_csv(file_path)
        label = data.iat[0,0]
        file_name = file_name.replace("label", "data")
        labels[file_name] = label if label != 10 else 7

    return labels

labels_kaggle = get_labels(imu_label_path)

In [4]:
### TRAINING

def train_model(dataloader, model, criterion, optimiser, epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        
        for batch_features, batch_labels in dataloader:
            if isinstance(model, CNN1DClassifier):
                batch_features = batch_features.transpose(1, 2)
            batch_features = batch_features.to(device)
            batch_labels = batch_labels.to(device).long()
            
            optimiser.zero_grad()
            outputs = model(batch_features)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimiser.step()
            
            total_loss += loss.item()
            
        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")

statistical_processing = False
dataloader = create_dataloader(WindowedCSVDataset, imu_data_path, labels_kaggle, statistical_processing, batch_size=5)

# model_kaggle = LSTMClassifier(input_size=44*3 if statistical_processing else 44, hidden_size=64, num_layers=2, num_classes=8)
# model_kaggle = MLPClassifier(input_size=25*44, num_classes=8)     # window_size * features
model_kaggle = CNN1DClassifier(input_size=44, num_classes=8)

lr = 0.001
batch_size = 16
epochs = 20
criterion = nn.CrossEntropyLoss()
optimiser = optim.Adam(model_kaggle.parameters(), lr)

train_model(dataloader, model_kaggle, criterion, optimiser, epochs)

Epoch 1/20 - Loss: 0.4682
Epoch 2/20 - Loss: 0.1904
Epoch 3/20 - Loss: 0.1305
Epoch 4/20 - Loss: 0.0962
Epoch 5/20 - Loss: 0.0805
Epoch 6/20 - Loss: 0.0668
Epoch 7/20 - Loss: 0.0498
Epoch 8/20 - Loss: 0.0516
Epoch 9/20 - Loss: 0.0505
Epoch 10/20 - Loss: 0.0338
Epoch 11/20 - Loss: 0.0412
Epoch 12/20 - Loss: 0.0383
Epoch 13/20 - Loss: 0.0355
Epoch 14/20 - Loss: 0.0373
Epoch 15/20 - Loss: 0.0332
Epoch 16/20 - Loss: 0.0294
Epoch 17/20 - Loss: 0.0394
Epoch 18/20 - Loss: 0.0318
Epoch 19/20 - Loss: 0.0297
Epoch 20/20 - Loss: 0.0252


In [5]:
### PREDICTION AND EVALUATION

def get_windows_from_csv(file_path, window_size=25, stride=5, statistical_processing=False):
    df = pd.read_csv(file_path)
    x = torch.tensor(df.iloc[:, 1:].values, dtype=torch.float32)  # [T, F]
    T, F = x.shape

    if T < window_size:
        pad = torch.zeros(window_size - T, F)
        x = torch.cat([x, pad], dim=0)
        T = window_size

    windows = []
    for start in range(0, T - window_size + 1, stride):
        window = x[start:start + window_size]  # [25, 44]
        
        # Add Stats (Must match the Training Dataset logic exactly!)
        if statistical_processing:
            w_mean = window.mean(dim=0).unsqueeze(0).expand(window_size, -1)
            w_std = window.std(dim=0).unsqueeze(0).expand(window_size, -1)
            window = torch.cat([window, w_mean, w_std], dim=-1) # [25, 132]
            
        windows.append(window)

    return torch.stack(windows)  # [num_windows, 25, 132 or 44]

def predict_csv(model, windows):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    windows = windows.to(device)

    if isinstance(model, CNN1DClassifier):
        windows = windows.transpose(1,2)

    with torch.no_grad():
        logits = model(windows)             # [num_windows, num_classes]
        probs = torch.softmax(logits, dim=1) # [num_windows, num_classes]

        # Aggregate: mean across windows
        avg_probs = probs.mean(dim=0)        # [num_classes]
        pred_class = torch.argmax(avg_probs).item()
        confidence = torch.max(avg_probs).item()

    return pred_class, confidence

def evaluate_folder(model, folder_path, labels_dict, window_size=25, stride=5, statistical_processing=False):
    y_true = []
    y_pred = []

    for file_name, label in labels_dict.items():
        file_path = os.path.join(folder_path, file_name)
        windows = get_windows_from_csv(file_path, window_size, stride, statistical_processing)

        pred, confidence = predict_csv(model, windows)
        y_true.append(label)
        y_pred.append(pred)

    accuracy = accuracy_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred, labels=list(range(8)))
    report = classification_report(y_true, y_pred, labels=list(range(8)), zero_division=0)
    
    return accuracy, conf_matrix, report

acc, cm, report = evaluate_folder(model_kaggle, imu_data_path, labels_kaggle, statistical_processing=statistical_processing)
# RNN + LSTM (with mean, std dev): 1
# RNN + LSTM: 1
# 1D CNN: 0.992
# MLP: 0.990